In [26]:
from datetime import datetime
import math
import time
import tensorflow as tf

In [27]:
batch_size =32
num_batches=100

In [28]:
def print_activations(t):
    print(t.op.name,'',t.get_shape().as_list())

In [29]:
def inference(images):
    parameters = []
    with tf.name_scope('conv1') as scope:
        kernel = tf.Variable(tf.truncated_normal([11,11,3,64],dtype=tf.float32,stddev=1e-1),name='weights')
        conv = tf.nn.conv2d(images,kernel,[1,4,4,1],padding='SAME')
        biases=tf.Variable(tf.constant(0.0,shape=[64],dtype=tf.float32),trainable=True,name='biases')
        bias = tf.nn.bias_add(conv,biases)
        conv1=tf.nn.relu(bias,name=scope)
        print_activations(conv1)
        parameters+=[kernel,biases]
        lrn1=tf.nn.lrn(conv1,4,bias=1.0,alpha=0.001/9,beta=0.75,name='lrn1')
        pool1=tf.nn.max_pool(lrn1,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID',name='pool1')
        print_activations(pool1)
    with tf.name_scope('conv2') as scope:
        kernel =tf.Variable(tf.truncated_normal([5,5,64,192],dtype=tf.float32,stddev=1e-1),name='weights')
        conv = tf.nn.conv2d(pool1,kernel,[1,1,1,1],padding='SAME')
        biases = tf.Variable(tf.constant(0.0,shape=[192],dtype=tf.float32),trainable=True,name='biases')
        bias = tf.nn.bias_add(conv,biases)
        conv2=tf.nn.relu(bias,name=scope)
        parameters +=[kernel,biases]
        print_activations(conv2)
        lrn2=tf.nn.lrn(conv2,4,bias=1.0,alpha=0.001/9,beta=0.75,name='lrn2')
        pool2=tf.nn.max_pool(lrn2,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID',name='pool2')
        print_activations(pool2)
    with tf.name_scope('conv3') as scope:
        kernel = tf.Variable(tf.truncated_normal([3,3,192,384],dtype=tf.float32,stddev=1e-1),name='weights')
        conv = tf.nn.conv2d(pool2,kernel,[1,1,1,1],padding='SAME')
        biases = tf.Variable(tf.constant(0.0,shape=[384],dtype=tf.float32),trainable=True,name='biases')
        bias=tf.nn.bias_add(conv,biases)
        conv3=tf.nn.relu(bias,name=scope)
        parameters += [kernel,biases]
        print_activations(conv3)
    with tf.name_scope('conv4') as scope:
        kernel = tf.Variable(tf.truncated_normal([3,3,384,256],dtype=tf.float32,stddev=1e-1),name='weights')
        conv = tf.nn.conv2d(conv3,kernel,[1,1,1,1],padding='SAME')
        biases = tf.Variable(tf.constant(0.0,shape=[256],dtype=tf.float32),trainable=True,name='biases')
        bias=tf.nn.bias_add(conv,biases)
        conv4=tf.nn.relu(bias,name=scope)
        parameters += [kernel,biases]
        print_activations(conv4)
    with tf.name_scope('conv5') as scope:
        kernel = tf.Variable(tf.truncated_normal([3,3,256,256],dtype=tf.float32,stddev=1e-1),name='weights')
        conv = tf.nn.conv2d(conv4,kernel,[1,1,1,1],padding='SAME')
        biases = tf.Variable(tf.constant(0.0,shape=[256],dtype=tf.float32),trainable=True,name='biases')
        bias=tf.nn.bias_add(conv,biases)
        conv5=tf.nn.relu(bias,name=scope)
        parameters += [kernel,biases]
        print_activations(conv5)
        pool5 = tf.nn.max_pool(conv5,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID',name='pool5')
        print_activations(pool5)
        return pool5,parameters

In [30]:
def time_tensorflow_run(session,target,info_string): 
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0
    for i in range(num_batches + num_steps_burn_in):
        start_time = time.time()
        _ = session.run(target)
        duration = time.time() - start_time
        if i >= num_steps_burn_in:
            if not i %10:
                print('%s : step %d , duration = %.3f'%(datetime.now(),i-num_steps_burn_in,duration))
                total_duration += duration
                total_duration_squared  +=duration * duration
    mn = total_duration/num_batches
    vr = total_duration_squared/num_batches-mn*mn
    sd= math.sqrt(vr)
    print('%s: %s across %d steps, %.3f +/- %.3f sec / batch' % (datetime.now(),info_string, num_batches,mn,sd))

In [31]:
def run_benchmark():
    with tf.Graph().as_default():
        image_size = 224
        images= tf.Variable(tf.random_normal([batch_size,
                                             image_size,
                                             image_size,3],
                                            dtype=tf.float32,
                                            stddev=1e-1))
        pool5,parameters = inference (images)
        init = tf.global_variables_initializer()
        sess=tf.Session()
        sess.run(init)
        time_tensorflow_run(sess,pool5,'Forward')
        objective = tf.nn.l2_loss(pool5)
        grad = tf.gradients(objective,parameters)
        time_tensorflow_run(sess,grad,'Forward-backward')

In [32]:
run_benchmark()

(u'conv1', '', [32, 56, 56, 64])
(u'conv1/pool1', '', [32, 27, 27, 64])
(u'conv2', '', [32, 27, 27, 192])
(u'conv2/pool2', '', [32, 13, 13, 192])
(u'conv3', '', [32, 13, 13, 384])
(u'conv4', '', [32, 13, 13, 256])
(u'conv5', '', [32, 13, 13, 256])
(u'conv5/pool5', '', [32, 6, 6, 256])
2017-09-08 21:50:19.342650 : step 0 , duration = 0.045
2017-09-08 21:50:19.797985 : step 10 , duration = 0.046
2017-09-08 21:50:20.253396 : step 20 , duration = 0.046
2017-09-08 21:50:20.708530 : step 30 , duration = 0.046
2017-09-08 21:50:21.163351 : step 40 , duration = 0.046
2017-09-08 21:50:21.618411 : step 50 , duration = 0.046
2017-09-08 21:50:22.073655 : step 60 , duration = 0.045
2017-09-08 21:50:22.529132 : step 70 , duration = 0.045
2017-09-08 21:50:22.984809 : step 80 , duration = 0.045
2017-09-08 21:50:23.439855 : step 90 , duration = 0.045
2017-09-08 21:50:23.849613: Forward across 100 steps, 0.005 +/- 0.014 sec / batch
2017-09-08 21:50:25.707332 : step 0 , duration = 0.140
2017-09-08 21:50:2